# 10-714 Homework 4 Extension

This homework is an extension of homework 4, where you will be implementing the Transformer architecture. For this assignment, all the things you need to implement is in the file `python/needle/nn/nn_transformer.py`. Other things in the needle library remains the same. This homework extension is built on homework 4, so make sure to copy the solutions from homework 4.

In [7]:
!make

-- Found pybind11: /opt/homebrew/include (found version "2.13.5")
CUDA_TOOLKIT_ROOT_DIR not found or specified
-- Could NOT find CUDA (missing: CUDA_TOOLKIT_ROOT_DIR CUDA_NVCC_EXECUTABLE CUDA_INCLUDE_DIRS CUDA_CUDART_LIBRARY) 
-- Configuring done (0.4s)
-- Generating done (0.0s)
-- Build files have been written to: /Users/david_bao/Desktop/SelfLearn/DL/10-414/hw4_extra/build
[100%] Built target ndarray_backend_cpu


In [1]:
%set_env PYTHONPATH ./python
%set_env NEEDLE_BACKEND nd

env: PYTHONPATH=./python
env: NEEDLE_BACKEND=nd


In [3]:
import sys
sys.path.append('./python')

In [10]:
# Download the PTB dataset

import urllib.request
import os

!mkdir -p './data/ptb'
# Download Penn Treebank dataset
ptb_data = "https://raw.githubusercontent.com/wojzaremba/lstm/master/data/ptb."
for f in ['train.txt', 'test.txt', 'valid.txt']:
    if not os.path.exists(os.path.join('./data/ptb', f)):
        urllib.request.urlretrieve(ptb_data + f, os.path.join('./data/ptb', f))

## Transformers

In the previous homework you have implemented two sequence models, the Recurrent Neural Network, and Long Short-Term Memory. These models were once the state-of-the-art and default architecture choices on sequence modelling tasks, including language generation, until recently when the famous paper "[Attention Is All You Need](https://arxiv.org/abs/1706.03762)" (Vaswani et al. 2017) came out in 2017. Since then, Transformers, a model architecture introduced in the aforementioned paper, have become the standard and most performant class of model on language tasks. 

You will be implementing a Transformer in `python/needle/nn/nn_transformer.py`.

Transformers are composed of three mains components that you will implement. 
1. A masked multi-head attention mechanism that adaptively focuses on different timesteps of a sequence. 
2. A residual block consisting of the attention layer followed by a two-layer neural network applied independently at each timestep. 
3. A Transformer model consisting of several stacked residual blocks (in this homework you will implement a decoder-only transformer).

![model](https://miro.medium.com/v2/1*ZCFSvkKtppgew3cc7BIaug.png)

The above is a photo of the Transformer architecture from Vaswani et al. 2017. The version of the transformer you will implement is nearly identical, but has layer normalization applied at the start of each residual block (referred to as a [prenorm variant](https://arxiv.org/abs/2002.04745) of the Transformer).

## Part 1: Implementing the Multi-Head Attention Activation Layer

In this subproblem, you will be implementing the `forward` function of a "base" attention activation layer `MultiHeadAttention` in `python/needle/nn/nn_transformer.py`. This activation layer will take in three inputs: 
<p style="text-align: center;">multi-head queries $Q \in R^\mathcal{B \times H \times T \times D}$, keys $K \in R^\mathcal{B \times H \times T \times D}$, and values $V \in R^\mathcal{B \times H \times T \times D}$</p>

where $B$ is the batch size, $H$ is the number of attention heads, $T$ is the sequence length, and $D$ is the hidden dimension. 

The attention output $X \in R^{B \times H \times T \times D}$ is computed as follows:

<p style="text-align: center;">$X = \text{softmax}(\frac{Q K^T}{\sqrt{D}}) V$</p>

Note that the matrix multiplications above are batched. This functionality is not natively supported in needle yet, so we have provided a convenient function `matmul` for batched matrix multiplications in `MultiHeadAttention`. Your goal in this section is to return $X$ given the input queries, keys, and values. 

For auto-regressive Transformer, this attention should support causal masking using the function `self.create_causal_mask` we have provided. This is to make sure that the prediction of next token only depends on it's previous tokens. Specifically, causal masking is applying a mask before the softmax so that the softmax probability is computed over a masked matrix of $\frac{Q K^T}{\sqrt{D}}$. 

In addition, your implementation should apply dropout to the attention softmax $\text{softmax}(\frac{Q K^T}{\sqrt{D}})$. You can use the `self.dropout` function of the `MultiHeadAttention` module.

Importantly, this layer is only an activation function, and has no trainable variables (these come later).

Once you have finished your implementation, test your code with the following test cases.

In [4]:
!python3 -m pytest -s -l -v -k "attention_activation"

============================= test session starts ==============================
platform darwin -- Python 3.12.3, pytest-8.3.3, pluggy-1.5.0 -- /opt/homebrew/anaconda3/envs/llm/bin/python3
cachedir: .pytest_cache
rootdir: /Users/david_bao/Desktop/SelfLearn/DL/10-414/hw4_extra
collecting ... Using needle backend
collected 112 items / 96 deselected / 16 selected                              

tests/hw4_extra/test_transformer.py::test_attention_activation[cpu-0.0-False-64-31-5-4] PASSED
tests/hw4_extra/test_transformer.py::test_attention_activation[cpu-0.0-False-64-31-5-8] PASSED
tests/hw4_extra/test_transformer.py::test_attention_activation[cpu-0.0-True-64-31-5-4] PASSED
tests/hw4_extra/test_transformer.py::test_attention_activation[cpu-0.0-True-64-31-5-8] PASSED
tests/hw4_extra/test_transformer.py::test_attention_activation[cpu-0.1-False-64-31-5-4] PASSED
tests/hw4_extra/test_transformer.py::test_attention_activation[cpu-0.1-False-64-31-5-8] PASSED
tests/hw4_extra/test_transformer.py::

In [ ]:
!python3 -m mugrade submit "YOUR_KEY_HERE" -k "attention_activation"

## Part 2 Implementing the Self-Attention Layer with trainable parameters

In this subproblem, you will use the `MultiHeadAttention` class you just implemented, and wrap it in a subclass of `Module` called `AttentionLayer` in `python/needle/nn/nn_transformer.py`. 

This layer implements the self-attention with prenorm (when k, and v are None in the `self.forward` call) and cross-attention (when k and v are present in the `self.forward` call). We have provided skeleton code with the appropriate layer attributes defined. Your job is to write the forward pass of the `AttentionLayer`. Note that you are implementing multi-head attention, where the number of attention heads is given by the `self.num_head` attribute of the `AttentionLayer` class.

Given inputs $Q \in R^\mathcal{B \times T \times D'}$, keys $K \in R^\mathcal{B \times T \times D'}$, and values $V \in R^\mathcal{B \times T \times D'}$ where $B$ is the batch size, $T$ is the sequence length, and $D'$ is the embedding dimension. This layer performs the following computation sequentially:

(1) map queries, key, and values to heads.

<p style="text-align: center;">$Q' = \text{LayerNorm}_q (Q) \; W_q$</p>

<p style="text-align: center;">$K' = \text{LayerNorm}_k (K) \; W_k$</p>

<p style="text-align: center;">$V' = \text{LayerNorm}_v (V) \; W_v$</p>

where $\text{LayerNorm}_q , \text{LayerNorm}_k, \text{LayerNorm}_v $ are the prenorm `self.prenorm_q`, `self.prenorm_k` and `self.prenorm_v` respectively.

(2) unravel heads from the channels axis.

<p style="text-align: center;">$Q' \in R^{B \times T \times (HD)} \to Q' \in R^{B \times H \times T \times D} $</p>

<p style="text-align: center;">$K' \in R^{B \times T \times (HD)} \to K' \in R^{B \times H \times T \times D} $</p>

<p style="text-align: center;">$V' \in R^{B \times T \times (HD)} \to V' \in R^{B \times H \times T \times D} $</p>

where $H$ and $D$ are `self.num_head` and `self.head_dim` respectively.

(3) compute the multi-head attention activation.

<p style="text-align: center;">$X = \text{softmax}(\frac{Q' (K')^T}{\sqrt{D}}) V'$</p>

<p style="text-align: center;">$X \in R^{B \times H \times T \times D} \to X \in R^{B \times T \times H \times D} $</p>

<p style="text-align: center;">$X \in R^{B \times T \times H \times D} \to X \in R^{B \times T \times (HD)}$</p>

The last two steps do a transpose and then reshape to get the hidden states to be the correct shape.

(4) project back to the input space of the layer with `self.out_projection`

<p style="text-align: center;">$X' = X \; W_o$</p>

Your goal in this part is to return $X$ in the `self.forward` call of `AttentionLayer`. For debugging, you may capture the `probs` variable returned by the inner `MultiHeadAttention` module and store it in an attribute such as `self.probs` of the attention layer.

Once finished, you may test your layer with the following test cases.

In [14]:
!python3 -m pytest -s -l -v -k "attention_layer"

============================= test session starts ==============================
platform darwin -- Python 3.12.3, pytest-8.3.3, pluggy-1.5.0 -- /opt/homebrew/anaconda3/envs/llm/bin/python3
cachedir: .pytest_cache
rootdir: /Users/david_bao/Desktop/SelfLearn/DL/10-414/hw4_extra
collecting ... Using needle backend
collected 112 items / 80 deselected / 32 selected                              

tests/hw4_extra/test_transformer.py::test_attention_layer[cpu-0.0-False-32-8-27-5-4] PASSED
tests/hw4_extra/test_transformer.py::test_attention_layer[cpu-0.0-False-32-8-27-5-8] PASSED
tests/hw4_extra/test_transformer.py::test_attention_layer[cpu-0.0-False-32-8-27-11-4] PASSED
tests/hw4_extra/test_transformer.py::test_attention_layer[cpu-0.0-False-32-8-27-11-8] PASSED
tests/hw4_extra/test_transformer.py::test_attention_layer[cpu-0.0-True-32-8-27-5-4] PASSED
tests/hw4_extra/test_transformer.py::test_attention_layer[cpu-0.0-True-32-8-27-5-8] PASSED
tests/hw4_extra/test_transformer.py::test_attention_l

In [ ]:
!python3 -m mugrade submit "YOUR_KEY_HERE" -k "attention_layer"

## Part 3 Implementing a prenorm residual Transformer Layer

You now have all the parts necessary to build a full Transformer by this point. In this subproblem, you will assemble the attention layer with a feedforward network into a stackable residual block. We have provided starter code in the `TransformerLayer` class. 

You will need to define the necessary class attributes in the `self.__init__` call of the module `TransformerLayer`, and fill in the forward pass in `self.forward`. Your transformer layer should support dropout applied to $X'$ from the previous step before adding a residual connection. Implement the following pseudocode of the layer, properly handling the intermediate tensor shapes:

x - current sequence of hidden states

<p style="text-align: center;">$x = x + \text{Dropout}(\text{Attention}(x))$</p>
<p style="text-align: center;">$x = x + \text{Dropout}(\text{Linear}_{2}(\text{Dropout}(\text{ReLU}(\text{Linear}_{1}(\text{LayerNorm1d}(x))))))$</p>

For the MLP, there are two Linear layers $\text{Linear}_{1}$ and $\text{Linear}_{2}$:
- $\text{Linear}_{1}$: input shape `q_features`, output shape `hidden_size`
- $\text{Linear}_{2}$: input shape `hidden_size`, output shape `q_features`

Once finished, run the following test cases.

In [17]:
!python3 -m pytest -s -l -v -k "transformer_layer"

============================= test session starts ==============================
platform darwin -- Python 3.12.3, pytest-8.3.3, pluggy-1.5.0 -- /opt/homebrew/anaconda3/envs/llm/bin/python3
cachedir: .pytest_cache
rootdir: /Users/david_bao/Desktop/SelfLearn/DL/10-414/hw4_extra
collecting ... Using needle backend
collected 112 items / 80 deselected / 32 selected                              

tests/hw4_extra/test_transformer.py::test_transformer_layer[cpu-0.0-False-64-32-8-27-5-2] PASSED
tests/hw4_extra/test_transformer.py::test_transformer_layer[cpu-0.0-False-64-32-8-27-5-4] PASSED
tests/hw4_extra/test_transformer.py::test_transformer_layer[cpu-0.0-False-64-32-8-27-11-2] PASSED
tests/hw4_extra/test_transformer.py::test_transformer_layer[cpu-0.0-False-64-32-8-27-11-4] PASSED
tests/hw4_extra/test_transformer.py::test_transformer_layer[cpu-0.0-True-64-32-8-27-5-2] PASSED
tests/hw4_extra/test_transformer.py::test_transformer_layer[cpu-0.0-True-64-32-8-27-5-4] PASSED
tests/hw4_extra/test_tr

In [106]:
!python3 -m mugrade submit "YOUR_KEY_HERE" -k "transformer_layer"

submit
============================= test session starts ==============================
platform darwin -- Python 3.12.3, pytest-8.3.3, pluggy-1.5.0
rootdir: /Users/david_bao/Desktop/SelfLearn/DL/10-414/hw4_extra
collecting ... Using needle backend
collected 4 items / 3 deselected / 1 selected                                  

tests/hw4_extra/test_transformer.py F

=================================== FAILURES ===================================
___________________________ submit_transformer_layer ___________________________

pyfuncitem = <Function submit_transformer_layer>

    @pytest.hookimpl(hookwrapper=True)
    def pytest_pyfunc_call(pyfuncitem):
        ## prior to test, initialize submission
        global _values, _submission_key, _errors
        _values = []
        _errors = 0
        func_name = pyfuncitem.name[7:]
        if os.environ["MUGRADE_OP"] == "submit":
>           _submission_key = start_submission(func_name)

/opt/homebrew/anaconda3/envs/llm/lib/python3.12/site-

## Part 4 Implementing the Transformer model

In this subsection, you will compose the residual transformer layers you implemented in the previous part to build the full Transformer model. Fill in the code in the `Transformer` class by defining a set of `num_layers` `TransformerLayer` modules with the appropriat parameters passed in from the parent `Transformer` class. Then, implement the `self.forward` call of the `Transformer`. 

As is, your current Transformer layers are permutation-invariant, and cannot tell which position each token is in the sequence. To break this symmetry, you will add a positional embedding to your Transformer.

The original Transformer paper uses sinusoidal positional embeddings, and then adds to the input embeddings before the first `TransformerLayer`. These work well, but a more common strategy in modern Transformers is to learn the positional embeddings. 

To do this, you should use `needle.nn.Embedding`. In your Transformer implementation, create a learnable positional encoding using `needle.nn.Embedding` from homework 4, with `num_embeddings` set as `sequence_len`. Given an input sequence, you should create a tensor that has the timestep id of each token in the sequence (timesteps have increasing value, representing the position of a token in time), and use it like a word id. 

Last, add the created positional encoding to the input token embeddings before your transformer layers.

Once complete, submit the following test cases.

In [21]:
!python3 -m pytest -l -v -k "transformer_model"

============================= test session starts ==============================
platform darwin -- Python 3.12.3, pytest-8.3.3, pluggy-1.5.0 -- /opt/homebrew/anaconda3/envs/llm/bin/python3
cachedir: .pytest_cache
rootdir: /Users/david_bao/Desktop/SelfLearn/DL/10-414/hw4_extra
collected 112 items / 80 deselected / 32 selected                              

tests/hw4_extra/test_transformer.py::test_transformer_model[cpu-0.0-False-32-8-2-64-27-5-8] PASSED [  3%]
tests/hw4_extra/test_transformer.py::test_transformer_model[cpu-0.0-False-32-8-2-64-27-11-8] PASSED [  6%]
tests/hw4_extra/test_transformer.py::test_transformer_model[cpu-0.0-False-32-8-4-64-27-5-8] PASSED [  9%]
tests/hw4_extra/test_transformer.py::test_transformer_model[cpu-0.0-False-32-8-4-64-27-11-8] PASSED [ 12%]
tests/hw4_extra/test_transformer.py::test_transformer_model[cpu-0.0-True-32-8-2-64-27-5-8] PASSED [ 15%]
tests/hw4_extra/test_transformer.py::test_transformer_model[cpu-0.0-True-32-8-2-64-27-11-8] PASSED [ 18%]
test

In [ ]:
!python3 -m mugrade submit "YOUR_KEY_HERE" -k "transformer_model"

Now, you can train a Transformer language model on the Penn Treebank dataset:

Note: make sure to initialize a transformer model in the class `LanguageModel` of `apps/models.py`; also for Transformers, the final linear head `self.linear` should take in input dimension `embedding_size`.

In [1]:
%set_env PYTHONPATH ./python
%set_env NEEDLE_BACKEND nd

env: PYTHONPATH=./python
env: NEEDLE_BACKEND=nd


In [2]:
import sys
sys.path.append('./python')

import needle as ndl
sys.path.append('./apps')
from models import LanguageModel
from simple_ml import train_ptb, evaluate_ptb

device = ndl.cpu()
corpus = ndl.data.Corpus("data/ptb")
train_data = ndl.data.batchify(corpus.train, batch_size=256, device=device, dtype="float32")
model = LanguageModel(20, len(corpus.dictionary), hidden_size=32, num_layers=1, seq_model='transformer', seq_len=20, device=device)
train_ptb(model, train_data, seq_len=20, n_epochs=10, device=device, lr=0.003, optimizer=ndl.optim.Adam)
evaluate_ptb(model, train_data, seq_len=20, device=device)

Using needle backend


  1%|          | 2/182 [00:30<45:38, 15.22s/it]


KeyboardInterrupt: 